# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [ ]:
# Import your libraries:

%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

# Challenge 1 - Import and Describe the Dataset

In this lab, we will use a dataset containing information about customer preferences. We will look at how much each customer spends in a year on each subcategory in the grocery store and try to find similarities using clustering.

The dataset is located [here](https://drive.google.com/file/d/1z1gYSD32ktbHuKSzB5JVS_u4YsLibh5F/view?usp=sharing), please download it and place it in the data folder.

In [ ]:
# loading the data:
customer = pd.read_csv('../data/Wholesale_customer_data.csv')
 
customer.head()

#### Explore the dataset with mathematical and visualization techniques. What do you find?

Checklist:

* What does each column mean?
* Any categorical data to convert?
* Any missing data to remove?
* Column collinearity - any high correlations?
* Descriptive statistics - any outliers to remove?
* Column-wise data distribution - is the distribution skewed?
* Etc.

Additional info: Over a century ago, an Italian economist named Vilfredo Pareto discovered that roughly 20% of the customers account for 80% of the typical retail sales. This is called the [Pareto principle](https://en.wikipedia.org/wiki/Pareto_principle). Check if this dataset displays this characteristic.

In [ ]:
# Your code here:
customer.dtypes
# data type is correct

#columns channel and region seem categorical (option o 1,2 or 3)
# while the rest of the columns are the amount spent

In [ ]:
# checking for missing data
null_cols = customer.isna().sum()
null_cols[null_cols > 0]

# no missing data found

In [ ]:
# checking for Column collinearity
customer.corr()

#no, only one very high correlation of 0.92, but as they are quite different categories, I don't think
#it should be removed (if we want to observe total $ spent)

In [ ]:
# checking
customer.describe().transpose()

sns.pairplot(customer)

# If we focus on the middle line, we can observe how the data is distributed
# for all atributed, except region (which is categorical anyway) the data is skwed to the left
# there seem to be quite some outliers, but if could be due to specific shopping patterns,
# I dont feel familiar enough with the data to decide to remove them yet

In [ ]:
# checking for PARETO principle
customer.loc[:,'Total'] = customer[['Fresh', 'Milk', 'Grocery', 'Frozen','Detergents_Paper', 'Delicassen']].sum(axis=1)
customer.head()

In [ ]:
customer.shape #440 rows

# twenty % of the customers is 88
cust_20 = 440*.2

# so we will select the highest 88 totals
customer['Total'].nlargest(int(cust_20)).sum() / customer['Total'].sum()

# it seems here that the top 20% of the customers only have 42% of the total, 
# so the pareto principle does not apply
#Paolo: very good analysis

# Challenge 2 - Data Cleaning and Transformation

If your conclusion from the previous challenge is the data need cleaning/transformation, do it in the cells below. However, if your conclusion is the data need not be cleaned or transformed, feel free to skip this challenge. But if you do choose the latter, please provide rationale.

In [ ]:
# Your comment here
# data types are OK, there is no missing data, and I'm choosing to keep the outliers

# Challenge 3 - Data Preprocessing

One problem with the dataset is the value ranges are remarkably different across various categories (e.g. `Fresh` and `Grocery` compared to `Detergents_Paper` and `Delicassen`). If you made this observation in the first challenge, you've done a great job! This means you not only completed the bonus questions in the previous Supervised Learning lab but also researched deep into [*feature scaling*](https://en.wikipedia.org/wiki/Feature_scaling). Keep on the good work!

Diverse value ranges in different features could cause issues in our clustering. The way to reduce the problem is through feature scaling. We'll use this technique again with this dataset.

#### We will use the `StandardScaler` from `sklearn.preprocessing` and scale our data. Read more about `StandardScaler` [here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).

*After scaling your data, assign the transformed data to a new variable `customers_scale`.*

In [ ]:
# Your import here:

from sklearn.preprocessing import StandardScaler

# Your code here:

model = StandardScaler()
customers_scale = model.fit_transform(customer)
customers_scale = pd.DataFrame(customers_scale)
customers_scale

# Challenge 3 - Data Clustering with K-Means

Now let's cluster the data with K-Means first. Initiate the K-Means model, then fit your scaled data. In the data returned from the `.fit` method, there is an attribute called `labels_` which is the cluster number assigned to each data record. What you can do is to assign these labels back to `customers` in a new column called `customers['labels']`. Then you'll see the cluster results of the original data.

In [ ]:
# Your code here:
from sklearn.cluster import KMeans

customer_kmeans = KMeans().fit(customers_scale)
customer_kmeans

customer['labels'] = customer_kmeans.labels_

Count the values in `labels`.

In [ ]:
# Your code here:
customer['labels'].value_counts()

# there are 8 different clusters, because default is n = 8

# Challenge 4 - Data Clustering with DBSCAN

Now let's cluster the data using DBSCAN. Use `DBSCAN(eps=0.5)` to initiate the model, then fit your scaled data. In the data returned from the `.fit` method, assign the `labels_` back to `customers['labels_DBSCAN']`. Now your original data have two labels, one from K-Means and the other from DBSCAN.

In [ ]:
# Your code here
from sklearn.cluster import DBSCAN

customer_model = DBSCAN(eps=0.5).fit(customers_scale)
customer['labels_DBSCAN'] = customer_model.labels_

Count the values in `labels_DBSCAN`.

In [ ]:
# Your code here
customer['labels_DBSCAN'].value_counts()
# the largest group is -1, which is noise... ?
#Paolo: yes, exactly is not able to properly cluster those elements

# Challenge 5 - Compare K-Means with DBSCAN

Now we want to visually compare how K-Means and DBSCAN have clustered our data. We will create scatter plots for several columns. For each of the following column pairs, plot a scatter plot using `labels` and another using `labels_DBSCAN`. Put them side by side to compare. Which clustering algorithm makes better sense?

Columns to visualize:

* `Detergents_Paper` as X and `Milk` as y
* `Grocery` as X and `Fresh` as y
* `Frozen` as X and `Delicassen` as y

Visualize `Detergents_Paper` as X and `Milk` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
# Your code here:

fig, [ax1, ax2] = plt.subplots(1,2, figsize = (20,5))

ax1.scatter(x=customer['Detergents_Paper'], y=customer['Milk'], c=customer['labels'])
ax1.set_title('K-Means - 8 clusters')


ax2.scatter(x=customer['Detergents_Paper'], y=customer['Milk'], c=customer['labels_DBSCAN'])
ax2.set_title('DBSCAN - 6 clusters')


plt.show()


Visualize `Grocery` as X and `Fresh` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
# Your code here:
fig, [ax1, ax2] = plt.subplots(1,2, figsize = (20,5))

ax1.scatter(x=customer['Grocery'], y=customer['Fresh'], c=customer['labels'])
ax1.set_title('K-Means - 8 clusters')


ax2.scatter(x=customer['Grocery'], y=customer['Fresh'], c=customer['labels_DBSCAN'])
ax2.set_title('DBSCAN - 6 clusters')


plt.show()

Visualize `Frozen` as X and `Delicassen` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
# Your code here:
fig, [ax1, ax2] = plt.subplots(1,2, figsize = (20,5))

ax1.scatter(x=customer['Frozen'], y=customer['Delicassen'], c=customer['labels'])
ax1.set_title('K-Means - 8 clusters')


ax2.scatter(x=customer['Frozen'], y=customer['Delicassen'], c=customer['labels_DBSCAN'])
ax2.set_title('DBSCAN - 6 clusters')


plt.show()
#Paolo: nice visualizations, could add labels on axes

Let's use a groupby to see how the mean differs between the groups. Group `customers` by `labels` and `labels_DBSCAN` respectively and compute the means for all columns.

In [ ]:
# Your code here:
customer.groupby('labels').mean()


In [ ]:
customer.groupby('labels_DBSCAN').mean()


Which algorithm appears to perform better?

In [ ]:
# Your observations here
# its really hard to tell from the averages, but from the graphs I would say that K-means
# does a better job, especially at identifying all the outliers and classifying them
# then again, I'm only judging this from intuition, I'm not really sure how to measure it
#Paolo: yes, I agree plus dbscan has many instances of noise. You could play with the parameters of dbscan
# and Kmeans to see what changes. In Kmeans with current parameters (n=8) you have a cluster with only one
# element

# Bonus Challenge 2 - Changing K-Means Number of Clusters

As we mentioned earlier, we don't need to worry about the number of clusters with DBSCAN because it automatically decides that based on the parameters we send to it. But with K-Means, we have to supply the `n_clusters` param (if you don't supply `n_clusters`, the algorithm will use `8` by default). You need to know that the optimal number of clusters differs case by case based on the dataset. K-Means can perform badly if the wrong number of clusters is used.

In advanced machine learning, data scientists try different numbers of clusters and evaluate the results with statistical measures (read [here](https://en.wikipedia.org/wiki/Cluster_analysis#External_evaluation)). We are not using statistical measures today but we'll use our eyes instead. In the cells below, experiment with different number of clusters and visualize with scatter plots. What number of clusters seems to work best for K-Means?

In [ ]:
# Your code here

In [ ]:
# Your comment here

# Bonus Challenge 3 - Changing DBSCAN `eps` and `min_samples`

Experiment changing the `eps` and `min_samples` params for DBSCAN. See how the results differ with scatter plot visualization.

In [ ]:
# Your code here

In [ ]:
# Your comment here